# Problem Description
Wake county located in North Carolina is one of the fastest growing counties in the USA large influx of immigrants from other states and nationalities calling it their home.
The highly educated workforce with a good mix of young and middle aged population drives the economy of the state and opens up tremendous  possibilities for new businesses to thrive.North carolina has been influenced greatly by settlers from New England and Hispanic immigrants.North carolinais also famous for its barbecue and bacon but other cuisines are starting to becoming popular.
This opens up a possibility to create a recommender system for anyone who wants to open a new restaurant that can be successful in the area based on the information
1. Demographics and Income
2. Existing restaurants density in the area(To identify any untapped market)
3. Types of restaurants based on cuisines
4. Existing Restaurant quality based on reviews and food inspections
5. Farmer's Market around the various neighbourhood

Wake county has 12 Towns and  46 Communities and for this work we will obtain the following datasets from https://data-wake.opendata.arcgis.com/datasets
1. Wake county restaurants Datasets(csv)
2. Wake county food inspections datasets(csv)
3. Wake county demographic information(csv)
4. Farmers Market http://www.wakegov.com/humanservices/publichealth/information/promotion/Pages/farmersmarkets.aspx
The data from the above datasets will be read via pandas dataframe and cleaned up to classify Restaurants to Townships and and quality index score.Then we will be able to cluster the restaurants in the area and as well as correlate the quality of restaurants again based on cuisine to the neighbourhood which would potentially provide us insights on the type of restaurant that can be setup. We will be using Foursquare api to obtain restaurant reviews in the locality to do additional correlation that will strengthen the model.We will finally demonstrate the efficacy with vuisualisation plots and graphs that will aid in the final decision

## Recommendation Success Criteria
We will determine the success of the recommender system by providing potential locations it can be set up for maximum success

### Import relevant libraries


In [6]:
import pandas as pd
import numpy as np
import matplotlib as mlt
from bs4 import BeautifulSoup
import requests


### Raleigh Demographic Distribution

In [54]:
website_url=requests.get('https://en.wikipedia.org/wiki/Raleigh,_North_Carolina').text
soup=BeautifulSoup(website_url,'lxml')
DemoTable=soup.find_all('table')[3]
dfRal_demo=pd.DataFrame(columns=['Race','Year=2010','Year=1990','Year=1970','Year=1950'],index=[1])
rows=DemoTable.find_all('tr')
l = []
for tr in rows:
    td = tr.find_all('td')
    row = [tr.text.strip() for tr in td]
    l.append(row)
dfRal_demo=pd.DataFrame(l, columns=['Race','Year=2010','Year=1990','Year=1970','Year=1950'])

In [55]:
dfRal_demo.drop(0)

,Race,Year=2010,Year=1990,Year=1970,Year=1950
1,White,57.5%,69.2%,76.6%,72.7%
2,—Non-Hispanic whites,53.3%,68.4%,76.4%,n/a
3,Black or African American,29.3%,27.6%,22.7%,27.2%
4,Hispanic or Latino (of any race),11.4%,1.4%,0.6%,n/a
5,Asian,4.3%,2.5%,0.3%,0.1%


### Durham Demographic Distribution

In [64]:
durham_demo=pd.read_csv("C:\Python3\Coursera_Captstone\Durham_Demographic.csv")

In [65]:
durham_demo

,Year=2010,Race
0,42.45%,White
1,37.9%,Non-Hispanic White
2,40.96%,Black or African American
3,5.07%,Asian American
4,0.51%,Native American
5,0.07%,Native Hawaiian or Other Pacific Islander
6,8.28%,some other race
7,2.66%,two or more races
8,14.22%,Hispanic or Latino of any race


### Demographic Information of the Various Towns in Wakecounty

### Get Geolocation for each of the cities and towns within Wake county

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [7]:
Cities=['Durham,NC','Raleigh,NC']
Towns=['Angier,NC','Apex','Cary','fuquay varina','Garner','Holly Springs','Knightdale','Morrisville','Rolesville','Wakeforest','Wendell','Zebulon']
for city in Cities:
    geolocator = Nominatim(user_agent="CapstoneProject")
    location = geolocator.geocode(city)
    latitude = location.latitude
    longitude = location.longitude
    print('The geograpical coordinate of '+city+' are {}, {}.'.format(latitude, longitude))
    # create map of Toronto using latitude and longitude values
    # map_Toronto = folium.Map(location=[latitude, longitude], zoom_start=10)
for town in Towns:
    geolocator = Nominatim(user_agent="CapstoneProject")
    location = geolocator.geocode(town)
    latitude = location.latitude
    longitude = location.longitude
    print('The geograpical coordinate of '+town+' are {}, {}.'.format(latitude, longitude))


The geograpical coordinate of Durham,NC are 36.0181316, -78.8751582.
The geograpical coordinate of Raleigh,NC are 35.7803977, -78.6390989.
The geograpical coordinate of Angier,NC are 35.5071045, -78.7391813.
The geograpical coordinate of Apex are 35.7325352, -78.8505516.
The geograpical coordinate of Cary are 35.7883046, -78.7811964.
The geograpical coordinate of fuquay varina are 35.61551875, -78.8164514666885.
The geograpical coordinate of Garner are 35.7112642, -78.6141709.
The geograpical coordinate of Holly Springs are 35.6512655, -78.8336218.
The geograpical coordinate of Knightdale are 35.7878975, -78.4822938.
The geograpical coordinate of Morrisville are 35.824341, -78.8300321.
The geograpical coordinate of Rolesville are 35.9232862, -78.4573914.
The geograpical coordinate of Wakeforest are 39.162885, -77.2125309.
The geograpical coordinate of Wendell are 35.780987, -78.3697213.
The geograpical coordinate of Zebulon are 35.8243208, -78.3147199.
